### Data munging (General)

#### Importing libraries and source files

In [20]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [21]:
# --  Choose which file to import from below
extended_df_name = 'extended_fmeca.xlsx' # The whole fmeca - which has all the newly calculated columns
#req_calc_name = 'fmeca_req_calc.xlsx' # A slice of the fmeca with selected columns that require calculation. The order of columns is important here.
#correct_abts_name = 'correct_abts.xlsx' # A slice of the fmeca which requires abt corrections.
#correct_dim_name = 'correct_dim.xlsx' # A slice of fmeca which requires correct diminution levels

# -- File path to import from
file_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working/ProcessedData'

In [22]:
df = pd.read_excel(os.path.join(file_location,extended_df_name),sheet_name='Sheet1')

In [23]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Asbuilt,Min,Pit %,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,Flag_2,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,repair_formulate,inspInt_formulate,allowables,allowableCheck,calc_y_n_formulate,Anomlay ID,stiff_anom_combo
0,1,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD-CR-192,AW,49-50,LL15,LBHD,4200,100,FR.49,500.0,LL15,0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,NaN,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7,8.0,280.0,NaN,0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0,0.477692,Local,Med,Med,Y,Generalised corrosion,13.0,3.7,0.715385,Tank boundary plating,Less than 6mm remaining,Permanent repair,-,NaN,This is an area of Generalised corrosion. The ...,NaN,Poor,R,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Less than 6mm remaining,Permanent repair,error,0.2,correct dim level,Y,4SWBT-LBHD-CR-192,LL15_4SWBT-LBHD-CR-192


### Get the pass fail data from calc sheet

In [24]:
# --  Choose which file to import from below
calc_file = 'FSC-JAD-01-TEC-62-A1 MV LBHD Calcs AN.xlsm' # The whole fmeca - which has all the newly calculated columns

# -- File path to import from
file_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets'
file_path = os.path.join(file_location,calc_file)
# -- sheet name
calc_sheet_name = '4 Stress and buckling calcs'
calc_sheet_range = 'A5:IN92'

In [25]:

book = xw.Book(file_path) #.sheets[calc_sheet_name].range(calc_sheet_range)
sht = book.sheets[calc_sheet_name]
rng = sht.range(calc_sheet_range)
df2 = rng.options(pd.DataFrame, index=False, header=True).value

In [26]:
#df2 = df2.transpose()

In [27]:
#df2.iloc[[1,86]].transpose().reset_index(drop=True)
def operation1(df):
    calc = df.iloc[[86]].iloc[:,4:].transpose().reset_index(drop=False)
    calc.rename(columns={'index':'stiff_anom_combo',86:'pass_fail'},inplace=True)
    return calc

calc = operation1(df2)

In [28]:
pass_fail_dict = dict(zip(calc['stiff_anom_combo'],calc['pass_fail']))

#### Apply the pass fail results as column into the fmeca sheet

In [29]:
def PassFail(row):
    combo_name = row['stiff_anom_combo']
    try:
        return pass_fail_dict[combo_name]
    except:
        return 'No calc required'

In [30]:
df['pass_fail'] = df.apply(PassFail,axis=1)

#### Notes function

In [31]:
df.head(3)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Asbuilt,Min,Pit %,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,Flag_2,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,repair_formulate,inspInt_formulate,allowables,allowableCheck,calc_y_n_formulate,Anomlay ID,stiff_anom_combo,pass_fail
0,1,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD-CR-192,AW,49-50,LL15,LBHD,4200,100,FR.49,500.0,LL15,0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.7900,3.7,6.2100,0.477692,R,Long'l bhd,Panel,Plate,NaN,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7,8.0,280.0,NaN,0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0,0.477692,Local,Med,Med,Y,Generalised corrosion,13.0,3.7,0.715385,Tank boundary plating,Less than 6mm remaining,Permanent repair,-,NaN,This is an area of Generalised corrosion. The ...,NaN,Poor,R,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Less than 6mm remaining,Permanent repair,error,0.2,correct dim level,Y,4SWBT-LBHD-CR-192,LL15_4SWBT-LBHD-CR-192,N
1,2,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD-CR-193,AW,49-50,LL21,LBHD,4000,100,FR.49,50.0,LL21,0,AH,48.7,15.0,0.2,12.0,12.75,6.9,5.8,4.1,8.5,10.8,11.8,12.1,12.1,NaN,NaN,9.0125,4.1,5.9875,0.399167,R,Long'l bhd,Panel,Plate,NaN,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7,8.0,280.0,NaN,0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,5.0,100,0.399167,Local,Med,Med,Y,Generalised corrosion,15.0,4.1,0.726667,Tank boundary plating,Less than 6mm remaining,Permanent repair,-,NaN,This is an area of Generalised corrosion. The ...,NaN,Poor,R,LL21,abt correct,15.0,Generalised corrosion,Tank boundary plating,Less than 6mm remaining,Permanent repair,error,0.2,correct dim level,Y,4SWBT-LBHD-CR-193,LL21_4SWBT-LBHD-CR-193,N
2,3,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD-CR-194,AW,50-51,LL11,LBHD,15,15,FR.51,NaN,LL11,NaN,AH,0,12.0,0.2,9.6,10.20,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5000,5.5,6.5000,0.541667,R,Long'l bhd,Panel,Plate,NaN,Pitting,Long'l bhdPanelPlatePitting,8.0,7,8.0,448.0,NaN,0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,8.0,3.0,4.0,96.0,NaN,0,0.541667,Local,Med,Med,N,Pit,12.0,5.5,0.541667,Tank boundary plating,Less than 6mm remaining,Permanent repair,Scheduled tank inspection,NaN,Local pitting on Long'l bhd Plate found. Remai...,Structural repair (Pad weld),Good,R,LL11,abt correct,12.0,Pit,Tank boundary plating,Less than 6mm remaining,Permanent repair,Scheduled tank inspection,0.2,correct dim level,N,4SWBT-LBHD-CR-194,LL11_4SWBT-LBHD-CR-194,No calc required


In [32]:
df[df['Final Anomaly description']=='Pit']['Detail'].unique()

array(['Less than 6mm remaining', 'Between 6 - 10mm remaining',
       'Substantial or below substantial loss',
       'Any loss greater than renewable',
       'More than 10mm remaining and greater than renewable loss'],
      dtype=object)

In [36]:
def Notes(row):
    pass_calc = row['pass_fail']
    stiffener_combo = row['stiff_anom_combo']
    rwt = np.round(row['Renewal Thickness (mm)'],1)
    coating = row['Coating condition']
    if row['Local Structure'] == 'Plate':
        detail_structure = row['Local Structure']
    else:
        detail_structure = row['Detail Structure']
    anomaly_type = row['Final Anomaly description']
    detail = row['Detail']
    structural_component = row['Primary Structure']
    min_t = row['Min']


    if anomaly_type == 'Generalised corrosion':
        if pass_calc == 'N':
            return f'This is an area of {anomaly_type}. The area fails hand calculations ({stiffener_combo}) against the structural performance standard. However given the lack of evidence of overloading of the structure (no buckling, cracks or fractures reported), before agreeing on a way ahead, it is suggested to wait for the FEA analysis results that take into account such factors as local Metocean data.'
        elif pass_calc == 'Y':
            return f'This is an area of {anomaly_type}. The area passes hand calculations ({stiffener_combo}) against the structural performance standard.'
        elif pass_calc == 'No calc required':
            return f'This is an area of {anomaly_type}. The area does not require calculations.'
        else:
            return 'decide calc required or not'

    elif anomaly_type == 'Pit within generalised corrosion':
        return '### ---- Follow the guideline of LR on deciding Pit or Corrosion'
    elif anomaly_type == 'Pit':
        if detail == 'Substantial or below substantial loss':
            if coating == 'Good':
                return f"Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location is {min_t}mm at least. No calculation required as this is a local pit. The diminution does not exceed the 30% ({rwt}mm RWT) limit for pits. Coating is {coating}."

            elif coating == 'Fair' or coating == 'Poor':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. The diminution does not exceed the 30% ({rwt}mm RWT) limit for pits. Coating is displaying signs of degradation with some breakdown occurring. As such the recommendation is to renew the coatings in the vicinity of this anomaly (hence "Arrest and Monitor").'
            else:
                return 'define coating condition'
        elif detail == 'More than 10mm remaining and greater than renewable loss':
            if coating == 'Good':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. The diminution does not exceed the 30% ({rwt}mm RWT) limit for pits. Given the localised nature of the pit and the fact that 10mm of wall thickness remains as well as the fact that the coating is {coating}, it is suggested that “monitor only” be applied whilst a permanent repair (potentially in the form of pad welding) be considered.'
            elif coating == 'Fair' or coating == 'Poor':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. The diminution does not exceed the 30% ({rwt}mm RWT) limit for pits. Given the localised nature of the pit and the fact that 10mm of wall thickness remains as well as the fact that the coating is exhibiting some signs of breakdown, it is suggested that “arrest and monitor” be applied whilst a permanent repair (potentially in the form of pad welding) be considered.'
            else:
                return 'define coating condition'
        elif detail == 'Any loss greater than renewable':
            if coating == 'Good':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. The diminution does not exceed the 30% ({rwt}mm RWT) limit for pits. Coating is intact.'
            elif coating == 'Fair' or coating == 'Poor':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. The diminution does not exceed the 30% ({rwt}mm RWT) limit for pits. Coating is displaying signs of degradation with some breakdown occurring. As such the recommendation is to renew the coatings in the vicinity of this anomaly (hence "Arrest and Monitor").'
            else:
                return 'define coating condition'
        elif detail == 'Less than 6mm remaining': # Ask Kamaraj for the best notes for this condition
            if coating == 'Good':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. Coating is intact.'
            elif coating == 'Fair' or coating == 'Poor':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. Coating is displaying signs of degradation with some breakdown occurring.'
        elif detail == 'Between 6 - 10mm remaining': # Ask Kamaraj for the best notes for this condition
            if coating == 'Good':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. Coating is intact.'
            elif coating == 'Fair' or coating == 'Poor':
                return f'Local pitting on {structural_component} {detail_structure} found. Remaining thickness at pit location  is {min_t}mm at least. No calculation required as this is a local pit. Coating is displaying signs of degradation with some breakdown occurring. As such the recommendation is to renew the coatings in the vicinity of this anomaly (hence "Arrest and Monitor").'
        else:
            return 'define detail' 
    else:
        return 'define anomaly type'

In [37]:
df['formulated_notes'] = df.apply(Notes,axis=1)

In [38]:
xw.view(df)